# Auto-ML BCI

This notebook demos the usage of `automl_BCI` in `neurol.BCI`.

`automl_BCI` is a BCI class in `neurol.BCI` which trains a custom machine learning model at initialization by recording data associated with some choice of "brain states". After recording data for each brain state, it transforms the data using the given `transformer` and epochs the recordings. Finally, it trains the given `model` using that data and reports back the performance it achieves.

After this initialization step (performed by `build_model`), the `automl_BCI` runs much like a typical `neurol.BCI`.

In [1]:
from neurol.BCI import automl_BCI

In [2]:
help(automl_BCI)

Help on class automl_BCI in module neurol.BCI:

class automl_BCI(generic_BCI)
 |  automl_BCI(model, epoch_len, n_states, transformer=None, action=<built-in function print>)
 |  
 |  Implements a Brain-Computer Interface which builds its own classifier
 |  by training a machine learning model in the calibration stage.
 |  
 |  At calibration, data is recorded for some number of brain-states
 |  then a machine-learning classifier is trained on the transformed data.
 |  
 |  Internally manages a buffer of the signal given in `stream` and
 |  continuously performs classification on appropriately transformed
 |  real-time neural data. At each classification, a corresponding `action`
 |  is performed.
 |  
 |  Attributes:
 |      model: a machine learning model.
 |      classifier: the model's predictor after training.
 |          accepts transformed data and returns classification.
 |      transformer: function which takes in the most recent data (`buffer`)
 |          and returns the trans

As the documentation says, an `automl_BCI` object needs to be initialized with a `model`, `epoch_len`, `n_states`. 

The `model` is any model object with `fit(X, y)` and `predict(X)` methods. Make sure that this type of model is compatible with the data after transformation. 

`epoch_len` is the desired length of epochs in # of samples. This is used when epoching to train the model, as well as when running the BCI. This is what is passed in to the transformer before being passed to the model.

`n_states` is the number of brain states we are asking the model to distinguish between. Internally, the BCI identifies brain states by an index starting at zero. In `build_model`, `n_states` different recordings are made for training.

Finally, an `automl_BCI` object also needs a `transformer` and an `action` (like other `neurol.BCI` objects). 

In [3]:
# let's define our parameters here
epoch_len = 50 # (samples)
recording_length = 1 # (seconds)
n_states = 3

transformer = lambda x: x[:, 0] # only use first channel

from sklearn import svm
model = svm.SVC() # support vector machine


now we can define the `automl_BCI`

In [4]:
bci = automl_BCI(model, epoch_len, n_states, transformer=transformer, action=print)

As always, we get our stream of data...

In [5]:
from neurol.connect_device import get_lsl_EEG_inlets
import neurol.streams

inlets = get_lsl_EEG_inlets()
inlet = inlets[0]

stream = neurol.streams.lsl_stream(inlet)

Finally, we can initialize the BCI and have it build its model.

In [6]:
bci.build_model(stream, recording_length)

Recording for 1 seconds...
Recording for 1 seconds...
Recording for 1 seconds...
Done! 

Performance on training data: 
              precision    recall  f1-score   support

         0.0       0.55      0.71      0.62        17
         1.0       0.75      0.19      0.30        16
         2.0       0.50      0.71      0.59        17

    accuracy                           0.54        50
   macro avg       0.60      0.53      0.50        50
weighted avg       0.60      0.54      0.50        50

Performance on test data: 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         4
         1.0       0.00      0.00      0.00         5
         2.0       0.22      0.50      0.31         4

    accuracy                           0.15        13
   macro avg       0.07      0.17      0.10        13
weighted avg       0.07      0.15      0.09        13



This process gives allows the BCI to create its own classifier. We are given the model's performance on both training data, and testing data so we can evaluate the BCI before running it.

We are now ready to run the BCI. It runs exactly like a `generic_BCI`, continuously transforming and classifying the stream of data, and responding with its action.

In [7]:
try:
    bci.run(stream)
except KeyboardInterrupt:
    print('\n')
    print('QUIT BCI')

2.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
0.0
2.0
2.0
2.0
2.0
0.0
2.0
2.0
0.0
2.0
0.0
2.0
2.0
0.0
2.0
0.0
0.0
0.0
0.0
2.0
2.0
0.0
2.0
0.0
2.0
0.0
0.0
2.0
0.0
2.0
2.0
0.0
2.0
0.0
2.0
2.0
0.0
2.0
2.0
0.0
2.0
0.0
2.0
0.0
1.0
2.0
2.0
1.0
2.0
0.0
2.0
0.0
1.0
2.0
2.0
2.0
2.0
0.0
2.0
0.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
0.0
2.0
0.0
2.0
2.0
0.0
2.0
0.0
0.0
0.0
0.0


QUIT BCI


That's it! What we did above was just a demo (recording length too short, epoch_len too small, trivial transformer, etc...), but it should give you an idea of how `automl_BCI` works. In practice, you need more data (longer recording length) and a more thoughtfull transformer. For a lot of applications, frequency domain information such as the powers of EEG wavebands is more predictive than the raw time-domain signal,so pick a transformer accordingly :)